Нам нужно защитить данные клиентов страховой компании. Мы разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель мы не будем.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression


In [2]:
df = pd.read_csv('/datasets/insurance.csv')
display(df.describe())
display(df.head())
display(df.info())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

### Ответ:

Для того что бы зашифровать данные клиентов мы используем 2 матрицы размером (5 х 5). Одну мы сгененируем из случайных чисел стандартного нормального распределения, другая будет обратная от нее.  Когда нам нужно будет получить исходные значения мы можем умножить наши измененные данные на обратную матрицу.

Формула:

**Шифрование**

$ X_{new} = X · P $

**Дешифровка**

$ X = X_{new} · P^{-1} $

*Где:*

 - $ P $ - матрица из случайного нормального распределения

 - $ P^{-1} $ - обратная матрица 
 
 - $ X $  - матрица c признаками по нашим клиентам

 - $ X_{new} $  - зашифрованная матрица c признаками по нашим клиентам
 
### Обоснование:

Произведение матриц ассоциативно, а значит:

$
(A · B) · C = A · (B · C)
$

Произведение матрицы на обратную от этой матрицы даст нам еденичную матрицу.

$
P · P^{-1} = E_n 
$

*Где:*

 - $ P $ - матрица из случайного нормального распределения

 - $ P^{-1} $ - обратная матрица 

 - $ E_n $  - еденичная матрица
    
А произведение любой матрицы на еденичную матрицу даст ту же самую матрицу.

$ P · E_n = P $

Таким образом мы получаем:

$ X · (P · P^{-1}) = (X · P) · P^{-1} = X · E_n $

**А значит что при умножении исходной матрицы на другую матрицу и на обратную от этой другой матрицы мы сохраняем значения исходной матрицы.**

$ X_{new} = X · P $

$ X = X_{new} · P^{-1} $



## 3. Алгоритм преобразования

**Алгоритм**

Генерируем матрицу из случайного нормального распределения, и обратуню к ней. Данные с признаками по клиентам  умножаем на сгенерированную матрицу и используем данные в модели. Если нам понядобится дешефифровать эти данные мы умножим зашифрованную матрицу на обратную.

**Обоснование**

Здесь мы используем логику из п. 2 в котором мы доказали что матрица умноженныая последовательно на другую матрицу и обратную к ней не меняет значения матрицы.

## 4. Проверка алгоритма

In [3]:
features = df.drop(columns='Страховые выплаты').values
targets = df['Страховые выплаты'].values

model = LinearRegression()
model.fit(features, targets)
print(f'R2 for linear regression with origin features {model.score(features, targets):.2f}')

matrix = np.random.normal(0,1,(4,4))
matrix_inv = np.linalg.inv(matrix)

features_new = features @ matrix

model_new = LinearRegression()
model_new.fit(features_new, targets)
print(
    f'R2 for linear regression with encripted features'
    f' {model_new.score(features_new, targets):.2f}'
     )

R2 for linear regression with origin features 0.42
R2 for linear regression with encripted features 0.42


#  Вывод 

Мы использовали свойства умножения матриц для шифрования данных. В частности мы исользовали свойство, что произведение матриц ассоциативно. Плюс мы доказали что качество работы модели при таком способе шифрования сохраняется. 

-----